In [253]:
# from ucimlrepo import fetch_ucirepo

# # fetch dataset
# early_stage_diabetes_risk_prediction = fetch_ucirepo(id=529)

# # data (as pandas dataframes)
# X = early_stage_diabetes_risk_prediction.data.features
# y = early_stage_diabetes_risk_prediction.data.targets

In [254]:
import pandas as pd

In [255]:
from src.constants import RAW_DATA_FILE_DIR

In [256]:
df = pd.read_csv("../data/raw/raw.csv")

In [257]:
df = df.drop(columns=df.columns[0])

In [258]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder

In [259]:
df.head()

,age,gender,polyuria,polydipsia,sudden_weight_loss,weakness,polyphagia,genital_thrush,visual_blurring,itching,irritability,delayed_healing,partial_paresis,muscle_stiffness,alopecia,obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive


In [ ]:
ordinal_categories = (
    df.select_dtypes(exclude="int64").drop(columns=["class", "gender"]).columns.tolist()
)

In [261]:
cat = df.select_dtypes(exclude="int64")
num = df.select_dtypes(include="int64")

In [262]:
ord_cat = [["Yes", "No"]] * len(ordinal_categories)

In [263]:
cat_ordinal = Pipeline(steps=[("Ordinal Pipeline", OrdinalEncoder(categories=ord_cat))])

In [264]:
cat_ohe = Pipeline(steps=[("One Hot Encoder", OneHotEncoder(drop="first", sparse_output=False))])

In [265]:
label_enc = Pipeline(steps=[("Label Encoder", LabelEncoder())])

In [266]:
from sklearn.linear_model import LogisticRegression

In [267]:
trf = ColumnTransformer(
    transformers=[
        ("Categorical Ordinal Transformation", cat_ordinal, ordinal_categories),
        ("One Hot Transformation", cat_ohe, ["gender"]),
    ],
    remainder="passthrough",
)

In [268]:
trf

ColumnTransformer(remainder='passthrough',
                  transformers=[('Categorical Ordinal Transformation',
                                 Pipeline(steps=[('Ordinal Pipeline',
                                                  OrdinalEncoder(categories=[['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Yes',
                                                                              'No']]))]),
                                 ['polyuria', 'polydipsia',
                                  'sudden_weight_loss', 'weakness',
                                  'polyphagia', 'genital_thrush',
                                  'visual_blurring', 'itching', 'irritability',
                                  'delayed_healing', 'partial_paresis',
                                  'muscle_stiffness', 'alopecia', 'obesity']),
                                ('One Hot Transformation',
                                 Pipeline(steps=[('One Hot Encoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False))]),
                                 ['gender'])])

In [269]:
data = trf.fit_transform(df)
labels = trf.get_feature_names_out()

trf_df = pd.DataFrame(data, columns=labels)

In [270]:
le = LabelEncoder()
trf_df["remainder__class"] = le.fit_transform(trf_df["remainder__class"])

In [271]:
X = trf_df.drop(columns=trf_df.columns[-1])
y = trf_df["remainder__class"]

In [272]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [273]:
scaler = StandardScaler()

In [274]:
final_pipeline = Pipeline([("scaler", StandardScaler()), ("classifier", LogisticRegression())])

In [275]:
model = final_pipeline.fit(X_train, y_train)

In [283]:
pred = model.predict(X_test)

In [284]:
from box import ConfigBox

In [292]:
transformations = ConfigBox(model.named_steps)

In [296]:
from sklearn.metrics import accuracy_score, classification_report

In [297]:
accuracy_score(y_test, pred)

0.9418604651162791

In [299]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92        61
           1       0.95      0.95      0.95       111

    accuracy                           0.94       172
   macro avg       0.94      0.94      0.94       172
weighted avg       0.94      0.94      0.94       172

